In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [2]:
# 기본적인 읽기
df = pd.read_csv('./Data/final_food_data.csv')
df

,식품명,식품대분류명,에너지(kcal),탄수화물(g),단백질(g),지방(g),당류(g),식품중량
0,국밥_돼지머리,밥류,137,15.94,6.70,5.16,0.16,900g
1,국밥_순대국밥,밥류,75,10.38,3.17,2.28,0.17,900g
2,국밥_콩나물,밥류,52,10.93,1.45,0.24,0.00,780g
3,기장밥,밥류,166,36.77,3.44,0.57,0.00,200g
4,김밥,밥류,140,19.98,4.84,4.55,0.00,230g
...,...,...,...,...,...,...,...,...
3143,청포묵무침,생채·무침류,43,6.54,1.30,1.68,0.72,150ml
3144,치커리겉절이,생채·무침류,22,3.51,0.72,0.73,1.89,180ml
3145,파래무침_파래_무,생채·무침류,23,4.86,1.07,0.38,2.62,130ml
3146,짜장소스,"장류, 양념류",100,12.86,4.38,3.64,4.16,200ml


In [37]:
pd.set_option('display.max_columns', None)
print(df.columns)

Index(['식품명', '식품대분류명', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '당류(g)',
       '식품중량'],
      dtype='object')


In [5]:
# 카테고리 매핑
category_mapping = {
    '밥류': '밥',
    '면 및 만두류': '면',
    '빵 및 과자류': '빵',
    '샌드위치': '빵',
    '브런치': '브런치',
    '국 및 탕류': '국 및 찌개',
    '찌개 및 전골류': '국 및 찌개',
    '죽 및 스프류': '죽 및 스프',
    '구이류': '요리',
    '찜류': '요리',
    '전·적 및 부침류': '요리',
    '튀김류': '요리',
    '나물·숙채류': '반찬',
    '생채·무침류': '반찬',
    '김치류': '반찬',
    '젓갈류': '반찬',
    '장아찌·절임류': '반찬',
    '볶음류': '요리',
    '조림류': '요리',
    '유제품류 및 빙과류': '디저트',
    '음료 및 차류': '음료',
}

In [6]:
# '식품대분류명' 열 값 수정
df['식품대분류명'] = df['식품대분류명'].map(category_mapping).fillna(df['식품대분류명'])

# 매핑되지 않은 카테고리 필터링
df = df[df['식품대분류명'].isin(category_mapping.values())]
df

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_50904\2603083067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['식품대분류명'] = df['식품대분류명'].map(category_mapping).fillna(df['식품대분류명'])


,식품명,식품대분류명,에너지(kcal),탄수화물(g),단백질(g),지방(g),당류(g),식품중량
0,국밥_돼지머리,밥,137,15.94,6.70,5.16,0.16,900g
1,국밥_순대국밥,밥,75,10.38,3.17,2.28,0.17,900g
2,국밥_콩나물,밥,52,10.93,1.45,0.24,0.00,780g
3,기장밥,밥,166,36.77,3.44,0.57,0.00,200g
4,김밥,밥,140,19.98,4.84,4.55,0.00,230g
...,...,...,...,...,...,...,...,...
3141,오징어채무침_채소,반찬,113,13.52,11.61,1.81,6.93,90ml
3142,참나물겉절이_고춧가루,반찬,18,2.91,0.74,0.60,1.10,190ml
3143,청포묵무침,반찬,43,6.54,1.30,1.68,0.72,150ml
3144,치커리겉절이,반찬,22,3.51,0.72,0.73,1.89,180ml


In [7]:
# food_data.csv로 저장
df.to_csv('./Data/food_data.csv',
              index=False,              # 인덱스 제외
              encoding='utf-8-sig',     # 한글 깨짐 방지
              sep=',',                  # 구분자 지정
              na_rep='NaN')            # 결측값 표현 방식 지정

In [9]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'diet_admin',
    'password': 'pproject2024!',
    'database': 'diet_recommendation_db'
}

# CSV 파일 읽기
df = pd.read_csv('./Data/food_data.csv')

# 열 이름 변경
df_renamed = df.rename(columns={
    '식품명': 'name',
    '식품대분류명': 'category',
    '에너지(kcal)': 'calories',
    '탄수화물(g)': 'carbs',
    '단백질(g)': 'protein',
    '지방(g)': 'fat',
    '당류(g)': 'sugar',
    '식품중량': 'serving_size'
})

# MySQL 연결 및 데이터 삽입
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")
df_renamed.to_sql('foods', con=engine, if_exists='append', index=False)

3135